<a href="https://colab.research.google.com/github/SingularitySmith/PRUT-Transcriber/blob/main/PRUT_Transcriber4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup and Imports

In [ ]:
!nvidia-smi


## 1.  Setting of Runtime Environment to GPU (e.g. "T4 GPU")

import tensorflow as tf
import torch
import torch.nn as nn
import locale
import os
import gc

# GPU Availability Check
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')

# PyTorch Device Configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device == 'cpu':
    raise SystemError('PyTorch couldn’t find GPU')